In [66]:
from os import listdir
from os.path import isfile
from json import load
from shutil import rmtree
from shutil import copytree
from os import makedirs

In [67]:
PASTA = "requisicoes"    
ORDEM_DA_LUZ = 'ordem_da_luz'

In [68]:
total_respondidas = 0
total_sem_responder  = 0
for item in listdir(PASTA):
    caminho = f'{PASTA}/{item}'
    caminho_resposta = f'{caminho}/resposta.json'
    if isfile(caminho_resposta):
        total_respondidas+=1
    if not isfile(caminho_resposta):
        print("sem responder em: ", item)
        total_sem_responder+=1


print("total: ", total_respondidas + total_sem_responder)
print("total respondidas: ",total_respondidas)
print("total sem responder",total_sem_responder)


total:  61367
total respondidas:  61367
total sem responder 0


In [ ]:
def modifica_num(valor,quantidade_max):
    tamanho = len(str(valor))
    total_0 = quantidade_max - tamanho
    final = ''
    for i in range(0,total_0):
        final+='0'
    final+=str(valor)
    return final



Organizando a ordem


In [70]:
rmtree(ORDEM_DA_LUZ,ignore_errors=True)
makedirs(ORDEM_DA_LUZ)

listagem = listdir(PASTA)
for item in listagem:
    caminho = f'{PASTA}/{item}'
    caminho_resposta = f'{caminho}/resposta.json'
    if not isfile(caminho_resposta):
        continue

    with open(caminho_resposta,'r') as arq:
        respostas = load(arq)


    if not respostas.get('json_body'):
        continue

    start_segundos = modifica_num(respostas["luz_adiquirida"]["segundos"])
    start_nano = modifica_num(respostas["luz_adiquirida"]["nanosegundos"])

    start = f'{start_segundos}:{start_nano}'

    end = ''
    if respostas.get('luz_liberada'):
        end_segundos = modifica_num(respostas["luz_liberada"]["segundos"])
        end_nano = modifica_num(respostas["luz_liberada"]["nanosegundos"])
        end = f'{end_segundos}:{end_nano}'
        
    novo_caminho =  f'{ORDEM_DA_LUZ}/{start}=>{end}|{item}'
    copytree(caminho,novo_caminho)

        



In [78]:
clientes = {
    "1":{
        "saldo":0,
        "limite":100000,
    },
    "2":{
        "saldo":0,
        "limite":80000
    },
    "3":{
        "saldo":0,
        "limite":1000000
    },
    "4":{
        "saldo":0,
        "limite":10000000
    },
    "5":{
        "saldo":0,
        "limite":500000
    }
}
ids = clientes.keys()
listagem = listdir(ORDEM_DA_LUZ)
listagem.sort()

for item in listagem:

    caminho = f'{ORDEM_DA_LUZ}/{item}'
    caminho_request = f'{caminho}/request.json'
    with open(caminho_request,'r') as arq:
        request = load(arq)
    
    caminho_resposta = f'{caminho}/resposta.json'
    if not isfile(caminho_resposta):
        raise Exception('aaaaaaa')
    
    rota:str = request['rota']

    rota_dividida = rota.split('/')
    id_cliente = rota_dividida[2]
    if id_cliente not in ids:
        raise Exception("aaaaaaaa")
    
    cliente_atual = clientes[id_cliente]
    
    tipo_rota = rota_dividida[3]
    if tipo_rota != 'transacoes':
        continue

    json_body = request['json_body']

    if json_body['valor'] < 0:
        raise  Exception("não deveria passar inteiros negativos")

    if json_body['valor'].__class__ != int:
        raise  Exception("não deveria passar qualquer tipo de valor nã inteiro")

    if  not json_body.get('descricao'):
        raise Exception("nãp ha descricao")


    if len(json_body['descricao']) == 0 or len(json_body['descricao'])> 10:
        raise Exception("descrição com valores inválidos")



    if json_body['tipo'] == 'c':
        cliente_atual['saldo'] += json_body['valor']

    
    if json_body['tipo'] == 'd':
        saldo_e_limite = cliente_atual['saldo'] + cliente_atual['limite']
        if saldo_e_limite - json_body['valor'] < 0:
            continue
        cliente_atual['saldo'] -= json_body['valor']

    #print(
    #    f'item: {item} =>' +
    #    f'cliente: {id_cliente} => '+
    #    f'saldo =>{cliente_atual["saldo"]}'
    #    )
    caminho_resposta = f'{caminho}/resposta.json'
    with open(caminho_resposta,'r') as arq:
        resposta = load(arq)

    if resposta['json_body']['saldo'] != cliente_atual['saldo']:
        raise Exception(
            'quebrou em ',item,
            'saldo é',resposta['json_body']['saldo'],
            'enquanto deveria ser', cliente_atual['saldo']
            )
    

item: 1709706639:698706=>1709706639:699308|3 =>cliente: 1 => saldo =>-1
item: 1709706639:710740=>1709706639:711182|1 =>cliente: 1 => saldo =>-2
item: 1709706639:711358=>1709706639:713443|5 =>cliente: 1 => saldo =>-3
item: 1709706639:716380=>1709706639:718448|2 =>cliente: 1 => saldo =>-4
item: 1709706639:729190=>1709706639:755170|6 =>cliente: 1 => saldo =>-5
item: 1709706639:755191=>1709706639:755617|7 =>cliente: 1 => saldo =>-6
item: 1709706639:755719=>1709706639:756202|8 =>cliente: 1 => saldo =>-7
item: 1709706639:756873=>1709706639:757336|4 =>cliente: 1 => saldo =>-8
item: 1709706639:766000=>1709706639:768521|9 =>cliente: 1 => saldo =>-9
item: 1709706639:770911=>1709706639:771359|11 =>cliente: 1 => saldo =>-10
item: 1709706639:773039=>1709706639:773454|14 =>cliente: 1 => saldo =>-11
item: 1709706639:775472=>1709706639:775965|12 =>cliente: 1 => saldo =>-12
item: 1709706639:779716=>1709706639:780193|15 =>cliente: 1 => saldo =>-13
item: 1709706639:781442=>1709706639:782029|10 =>cliente:

Exception: ('quebrou em ', '1709706647:230052=>1709706647:230620|142', 'saldo é', -16347, 'enquanto deveria ser', -13719)

In [ ]:
clientes

{'1': {'saldo': -9, 'limite': 100000},
 '2': {'saldo': 0, 'limite': 80000},
 '3': {'saldo': 0, 'limite': 1000000},
 '4': {'saldo': 0, 'limite': 10000000},
 '5': {'saldo': 0, 'limite': 500000}}